In [43]:
import nilearn
from nilearn import datasets
from tqdm.notebook import tqdm
import numpy as np
from nilearn import plotting

from graphUtils import *

## 0.1. Loading Atlas

In [44]:
atlas = datasets.fetch_atlas_aal(data_dir="/ptmp/arassat/nilearn_data")
atlas_filename = atlas.maps
labels = atlas.labels
num_atlas_reg = len(labels)
print("The Atlas of choice has {} labels.".format(num_atlas_reg))

The Atlas of choice has 116 labels.


## 0.2. Loading connectivity matrices to numpy arrays

In [45]:
num_subjects_tot = 115

subjects_removed = [6, 13, 45, 64, 78, 80, 88, 98, 105, 108, 113]
subjects = [i for i in range(1, num_subjects_tot + 1)]

for i in subjects_removed:
    subjects.remove(i)

num_subjects = num_subjects_tot - len(subjects_removed)

data_OFF = np.zeros((num_atlas_reg, num_atlas_reg, num_subjects))
data_ON = np.zeros((num_atlas_reg, num_atlas_reg, num_subjects))
print(data_OFF.shape)

for i, subject in enumerate(subjects):
    if subject < 10:
        file_str_OFF = "/ptmp/arassat/Abel/connectivity_matrices_aal/00%d_OFF_aal_connectivity.csv" % (subject)
        file_str_ON = "/ptmp/arassat/Abel/connectivity_matrices_aal/00%d_ON_aal_connectivity.csv" % (subject)
        data_OFF[:, :, i-1] = np.genfromtxt(file_str_OFF,delimiter=',')[1:,1:]
        data_ON[:, :, i-1] = np.genfromtxt(file_str_ON,delimiter=',')[1:,1:]
    elif subject >= 10 and subject < 100:
        file_str_OFF = "/ptmp/arassat/Abel/connectivity_matrices_aal/0%d_OFF_aal_connectivity.csv" % (subject)
        file_str_ON = "/ptmp/arassat/Abel/connectivity_matrices_aal/0%d_ON_aal_connectivity.csv" % (subject)
        data_OFF[:, :, i-1] = np.genfromtxt(file_str_OFF,delimiter=',')[1:,1:]
        data_ON[:, :, i-1] = np.genfromtxt(file_str_ON,delimiter=',')[1:,1:]
    else:
        file_str_OFF = "/ptmp/arassat/Abel/connectivity_matrices_aal/%d_OFF_aal_connectivity.csv" % (subject)
        file_str_ON = "/ptmp/arassat/Abel/connectivity_matrices_aal/%d_ON_aal_connectivity.csv" % (subject)
        data_OFF[:, :, i-1] = np.genfromtxt(file_str_OFF,delimiter=',')[1:,1:]
        data_ON[:, :, i-1] = np.genfromtxt(file_str_ON,delimiter=',')[1:,1:]
    
    np.fill_diagonal(data_OFF[:,:,i-1],0)
    np.fill_diagonal(data_ON[:,:,i-1],0)

    # removing the negative correlation values
    data_OFF[:,:,i-1] = np.where(data_OFF[:,:,i-1] > 0, data_OFF[:,:,i-1], 0)
    data_ON[:,:,i-1] = np.where(data_ON[:,:,i-1] > 0, data_ON[:,:,i-1], 0)

data = {
    "Off" : data_OFF,
    "On"  : data_ON
}

(116, 116, 104)


## 0.3. Loading thresholded & null-hyposthesis networks

In [46]:
import bct
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Setting the proportional thresholds
# In T. Xu et al. thresholds = np.arange(0.1, 0.55, 0.01)
thresholds = np.arange(0.05, 0.5, 0.01)
thresholds_wei_test = np.arange(0.05, 0.50, 0.05)
num_thresholds = thresholds.size
num_thresholds_wei_test = thresholds_wei_test.size
# random seed for generating null-hypothesis networks
seed = 42

# network will be saved in dictionaries
thresholded_data = {
    "Off" : np.zeros((num_atlas_reg, num_atlas_reg, num_subjects, num_thresholds)),
    "On"  : np.zeros((num_atlas_reg, num_atlas_reg, num_subjects, num_thresholds))
}
null_data = {
    "Off"  :  np.zeros(thresholded_data["Off"].shape),
    "On"   :  np.zeros(thresholded_data["On"].shape)
}
"""
thresholded_data_dist = {
    "Off" : np.zeros((num_atlas_reg, num_atlas_reg, num_subjects, num_thresholds_wei_test)),
    "On"  : np.zeros((num_atlas_reg, num_atlas_reg, num_subjects, num_thresholds_wei_test))
}
null_data_dist = {
    "Off"  :  np.zeros(thresholded_data_dist["Off"].shape),
    "On"   :  np.zeros(thresholded_data_dist["On"].shape)
}
"""

In [47]:
# Binarized networks
data = np.load('thresholded_and_null_data.npz')
thresholded_data["Off"] = data['thresholded_data_Off']
thresholded_data["On"] = data['thresholded_data_On']
null_data["Off"] = data['null_data_Off']
null_data["On"] = data['null_data_On']

"""
# Weighted networks
data_dist = np.load('thresholded_data_dist.npz')
thresholded_data_dist["Off"] = data_dist["thresholded_data_Off_dist"]
thresholded_data_dist["On"] = data_dist["thresholded_data_On_dist"]
null_data_dist['Off'] = np.load('null_data_Off_dist.npz')['null_data_Off_dist']
null_data_dist['On']  = np.load('null_data_On_dist.npz')['null_data_On_dist']
"""

# when networks generated separately due to high wall time for generation (dense thresholding) 
"""
th_data = np.load('thresholded_data.npz')
null_data_Off = np.load('null_data_Off.npz')
null_data_On = np.load('null_data_On.npz')
thresholded_data["Off"] = thresholded_data['thresholded_data_Off']
thresholded_data["On"] = thresholded_data['thresholded_data_On']
null_data["Off"] = null_data_Off['null_data_Off']
null_data["On"] = null_data_On['null_data_On']
"""

'\nth_data = np.load(\'thresholded_data.npz\')\nnull_data_Off = np.load(\'null_data_Off.npz\')\nnull_data_On = np.load(\'null_data_On.npz\')\nthresholded_data["Off"] = thresholded_data[\'thresholded_data_Off\']\nthresholded_data["On"] = thresholded_data[\'thresholded_data_On\']\nnull_data["Off"] = null_data_Off[\'null_data_Off\']\nnull_data["On"] = null_data_On[\'null_data_On\']\n'

Shape Check:

In [48]:
print(thresholded_data["Off"].shape, null_data["Off"].shape, thresholded_data_dist["Off"].shape, null_data_dist['Off'].shape)

(116, 116, 104, 45) (116, 116, 104, 45) (116, 116, 104, 9) (116, 116, 104, 9)


# 1. Computing Network Measures from T. Xu et al.

In [49]:
network_measures_Off = {
    "norm_charact_path_length"  : np.zeros((num_thresholds, num_subjects)),
    "norm_clustering_coef"      : np.zeros((num_thresholds, num_subjects)),
    "small_worldness"           : np.zeros((num_thresholds, num_subjects)),
    "norm_global_eff"           : np.zeros((num_thresholds, num_subjects)),
    "norm_local_eff"            : np.zeros((num_thresholds, num_subjects)),
    "size_of_LCC"               : np.zeros((num_thresholds, num_subjects))
}

network_measures_On = {
    "norm_charact_path_length"  : np.zeros((num_thresholds, num_subjects)),
    "norm_clustering_coef"      : np.zeros((num_thresholds, num_subjects)),
    "small_worldness"           : np.zeros((num_thresholds, num_subjects)),
    "norm_global_eff"           : np.zeros((num_thresholds, num_subjects)),
    "norm_local_eff"            : np.zeros((num_thresholds, num_subjects)),
    "size_of_LCC"               : np.zeros((num_thresholds, num_subjects))
}

In [66]:
"""
network_measures_wei_Off = {
    "norm_charact_path_length"  : np.zeros((num_thresholds_wei_test, num_subjects)),
    "norm_clustering_coef"      : np.zeros((num_thresholds_wei_test, num_subjects)),
    "small_worldness"           : np.zeros((num_thresholds_wei_test, num_subjects)),
    "norm_global_eff"           : np.zeros((num_thresholds_wei_test, num_subjects)),
    "norm_local_eff"            : np.zeros((num_thresholds_wei_test, num_subjects)),
    "size_of_LCC"               : np.zeros((num_thresholds_wei_test, num_subjects))
}

network_measures_wei_On = {
    "norm_charact_path_length"  : np.zeros((num_thresholds_wei_test, num_subjects)),
    "norm_clustering_coef"      : np.zeros((num_thresholds_wei_test, num_subjects)),
    "small_worldness"           : np.zeros((num_thresholds_wei_test, num_subjects)),
    "norm_global_eff"           : np.zeros((num_thresholds_wei_test, num_subjects)),
    "norm_local_eff"            : np.zeros((num_thresholds_wei_test, num_subjects)),
    "size_of_LCC"               : np.zeros((num_thresholds_wei_test, num_subjects))
}
"""

In [51]:
centrality_measures_Off = {
    "clustering_coef"           : np.zeros((num_thresholds, num_subjects, num_atlas_reg)),
    "local_eff"                 : np.zeros((num_thresholds, num_subjects, num_atlas_reg)), 
    "degree"                    : np.zeros((num_thresholds, num_subjects, num_atlas_reg))
}
                                           
centrality_measures_On = {
    "clustering_coef"           : np.zeros((num_thresholds, num_subjects, num_atlas_reg)),
    "local_eff"                 : np.zeros((num_thresholds, num_subjects, num_atlas_reg)),
    "degree"                    : np.zeros((num_thresholds, num_subjects, num_atlas_reg))
}                                  

In [67]:
"""
centrality_measures_wei_Off = {
    "clustering_coef"           : np.zeros((num_thresholds_wei_test, num_subjects, num_atlas_reg)),
    "local_eff"                 : np.zeros((num_thresholds_wei_test, num_subjects, num_atlas_reg)), 
    "degree"                    : np.zeros((num_thresholds_wei_test, num_subjects, num_atlas_reg)) # i.e., strength of each node / region
}
                                           
centrality_measures_wei_On = {
    "clustering_coef"           : np.zeros((num_thresholds_wei_test, num_subjects, num_atlas_reg)),
    "local_eff"                 : np.zeros((num_thresholds_wei_test, num_subjects, num_atlas_reg)),
    "degree"                    : np.zeros((num_thresholds_wei_test, num_subjects, num_atlas_reg)) # i.e., strength of each node / region
}  
"""

## 1.1. Binarized Networks

In [31]:
for j in tqdm(range(num_thresholds)):     
    # normalized characteristic path and global efficiency
    charpath_network_Off, global_eff_network_Off = charpath_and_global_eff(thresholded_data["Off"][:, :, :, j], num_subjects)
    charpath_null_Off, global_eff_null_Off = charpath_and_global_eff(null_data["Off"][:, :, :, j], num_subjects)
    charpath_network_On, global_eff_network_On = charpath_and_global_eff(thresholded_data["On"][:, :, :, j], num_subjects)
    charpath_null_On, global_eff_null_On = charpath_and_global_eff(null_data["On"][:, :, :, j], num_subjects)

    network_measures_Off["norm_charact_path_length"][j][:] = charpath_network_Off / charpath_null_Off
    network_measures_On["norm_charact_path_length"][j][:] = charpath_network_On / charpath_null_On
    network_measures_Off["norm_global_eff"][j][:] = global_eff_network_Off / global_eff_null_Off
    network_measures_On["norm_global_eff"][j][:]= global_eff_network_On / global_eff_null_On
    
    # normalized clustering coefficient
    network_clustering_coef_Off = np.array([np.mean(bct.clustering_coef_bu(thresholded_data["Off"][:, :, i, j])) for i in range(num_subjects)])
    null_clustering_coef_Off = np.array([np.mean(bct.clustering_coef_bu(null_data["Off"][:, :, i, j])) for i in range(num_subjects)])
    network_clustering_coef_On = np.array([np.mean(bct.clustering_coef_bu(thresholded_data["On"][:, :, i, j])) for i in range(num_subjects)])
    null_clustering_coef_On = np.array([np.mean(bct.clustering_coef_bu(null_data["On"][:, :, i, j])) for i in range(num_subjects)])
    network_measures_Off["norm_clustering_coef"][j][:] = network_clustering_coef_Off / null_clustering_coef_Off
    network_measures_On["norm_clustering_coef"][j][:] = network_clustering_coef_On / null_clustering_coef_On

    # small-worldness
    network_measures_Off["small_worldness"][j][:] = np.array(network_measures_Off["norm_clustering_coef"][j]) / np.array(network_measures_Off["norm_charact_path_length"][j])
    network_measures_On["small_worldness"][j][:] = np.array(network_measures_On["norm_clustering_coef"][j]) / np.array(network_measures_On["norm_charact_path_length"][j]) 
    
    # Size of the Largest Connected Component (LCC)
    network_measures_Off["size_of_LCC"][j][:] = np.array([len(max(nx.connected_components(nx.from_numpy_array(thresholded_data["Off"][:, :, i, j])))) for i in range(num_subjects)])
    network_measures_On["size_of_LCC"][j][:] = np.array([len(max(nx.connected_components(nx.from_numpy_array(thresholded_data["On"][:, :, i, j])))) for i in range(num_subjects)])
    
    # Clustering coefficient saved for all nodes
    centrality_net_clust_Off = np.array([bct.clustering_coef_bu(thresholded_data["Off"][:, :, i, j]) for i in range(num_subjects)])
    centrality_net_clust_On = np.array([bct.clustering_coef_bu(thresholded_data["On"][:, :, i, j]) for i in range(num_subjects)])
    # circumventing the issue of having a 0 clustering coefficient for a given region by dividing by the mean clustering coefficent (over all regions) 
    # of the null-hypothesis graph for a given graph density and subject. 
    centrality_measures_Off["clustering_coef"][j][:][:] = centrality_net_clust_Off / null_clustering_coef_Off[:, np.newaxis]
    centrality_measures_On["clustering_coef"][j][:][:] = centrality_net_clust_On / null_clustering_coef_On[:, np.newaxis]
    
    # Degree for all nodes
    centrality_net_deg_Off = np.array([bct.degrees_und(thresholded_data["Off"][:, :, i, j]) for i in range(num_subjects)])
    centrality_net_deg_On = np.array([bct.degrees_und(thresholded_data["On"][:, :, i, j]) for i in range(num_subjects)])
    null_mean_deg_Off = np.array([np.mean(bct.degrees_und(null_data["Off"][:, :, i, j])) for i in range(num_subjects)])
    null_mean_deg_On = np.array([np.mean(bct.degrees_und(null_data["On"][:, :, i, j])) for i in range(num_subjects)])
    centrality_measures_Off["degree"][j][:][:] = centrality_net_deg_Off / null_mean_deg_Off[:, np.newaxis]
    centrality_measures_On["degree"][j][:][:] = centrality_net_deg_On / null_mean_deg_On[:, np.newaxis]

  0%|          | 0/45 [00:00<?, ?it/s]

In [42]:
print(type(centrality_measures_Off["clustering_coef"][0]))
print(type(network_measures_Off["small_worldness"][0]))
print(type(network_measures_Off["norm_clustering_coef"][j]), type(network_measures_Off["norm_charact_path_length"][j]))
print(type(network_clustering_coef_On), type(null_clustering_coef_On))
print(network_clustering_coef_On.shape, null_clustering_coef_On.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(104,) (104,)


In [7]:
"""
local_eff_network_Off = np.array([local_eff(thresholded_data["Off"][:, :, :, j], num_subjects) for j in tqdm(range(num_thresholds))])
# Save the array to a text file
np.savetxt('local_eff_network_Off.txt', local_eff_network_Off)
# local_eff_network_Off = np.loadtxt('local_eff_network_Off.txt')
"""

  0%|          | 0/45 [00:00<?, ?it/s]

In [ ]:
local_eff_network_On = np.array([local_eff(thresholded_data["On"][:, :, :, j], num_subjects) for j in tqdm(range(num_thresholds))])
np.savetxt('local_eff_network_On.txt', local_eff_network_On)

  0%|          | 0/45 [00:00<?, ?it/s]

In [24]:
# Modify if the lists above can be saved correctly

local_eff_network_Off = np.loadtxt('local_eff_network_Off.txt')
local_eff_network_On = np.loadtxt('local_eff_network_On.txt')

for j in tqdm(range(num_thresholds)): 
    local_eff_null_Off = local_eff(null_data["Off"][:, :, :, j], num_subjects)
    local_eff_null_On = local_eff(null_data["On"][:, :, :, j], num_subjects)
    network_measures_Off["norm_local_eff"][j][:] = local_eff_network_Off[j] / local_eff_null_Off
    network_measures_On["norm_local_eff"][j][:] = local_eff_network_On[j] / local_eff_null_On

  0%|          | 0/46 [00:00<?, ?it/s]

/tmp/ipykernel_4906/3439679833.py:4: RuntimeWarning: invalid value encountered in true_divide
  network_measures_Off["norm_local_eff"][j][:] = local_eff_network_Off / local_eff_null_Off
/tmp/ipykernel_4906/3439679833.py:5: RuntimeWarning: invalid value encountered in true_divide
  network_measures_On["norm_local_eff"][j][:] = local_eff_network_On / local_eff_null_On


In [25]:
for j in tqdm(range(num_thresholds)): 
    # Local efficiency for all nodes (binarized networks)
    centrality_net_local_eff_Off = np.array([[local_efficiency_node(nx.from_numpy_array(thresholded_data["Off"][:, :, i, j]), node) 
                                              for node in nx.from_numpy_array(thresholded_data["Off"][:, :, i, j]).nodes ] 
                                              for i in range(num_subjects)])
    centrality_net_local_eff_On = np.array([[local_efficiency_node(nx.from_numpy_array(thresholded_data["On"][:, :, i, j]), node) 
                                              for node in nx.from_numpy_array(thresholded_data["On"][:, :, i, j]).nodes] 
                                              for i in range(num_subjects)])
    centrality_measures_Off["local_eff"][j][:][:] = centrality_net_local_eff_Off / local_eff_null_Off[:, np.newaxis]
    centrality_measures_On["local_eff"][j][:][:] = centrality_net_local_eff_On / local_eff_null_On[:, np.newaxis]                                            

  0%|          | 0/46 [00:00<?, ?it/s]

/tmp/ipykernel_4906/4001460178.py:9: RuntimeWarning: invalid value encountered in true_divide
  centrality_measures_Off["local_eff"][j][:][:] = centrality_net_local_eff_Off / local_eff_null_Off[:, np.newaxis]
/tmp/ipykernel_4906/4001460178.py:10: RuntimeWarning: invalid value encountered in true_divide
  centrality_measures_On["local_eff"][j][:][:] = centrality_net_local_eff_On / local_eff_null_On[:, np.newaxis]


## 1.2. Weighted Networks

In [68]:
"""
# Computing the global network measures for weighted graph
# for now not computing any null hypothesis metrics as done in Skidmore 2011
for j in tqdm(range(num_thresholds_wei_test)): 
    
    # normalized characteristic path and global efficiency
    charpath_network_Off, global_eff_network_Off = charpath_and_global_eff_wei(thresholded_data_dist["Off"][:, :, :, j], num_subjects)
    charpath_null_Off, global_eff_null_Off = charpath_and_global_eff(null_data_dist["Off"][:, :, :, j], num_subjects)
    charpath_network_On, global_eff_network_On = charpath_and_global_eff(thresholded_data_dist["On"][:, :, :, j], num_subjects)
    charpath_null_On, global_eff_null_On = charpath_and_global_eff(null_data_dist["On"][:, :, :, j], num_subjects)

    network_measures_wei_Off["norm_charact_path_length"][j][:] = charpath_network_Off / charpath_null_Off
    network_measures_wei_On["norm_charact_path_length"][j][:] = charpath_network_On / charpath_null_On
    network_measures_wei_Off["norm_global_eff"][j][:] = global_eff_network_Off / global_eff_null_Off
    network_measures_wei_On["norm_global_eff"][j][:]= global_eff_network_On / global_eff_null_On
    
    # normalized clustering coefficient
    network_clustering_coef_Off = np.array([np.mean(bct.clustering_coef_wu(thresholded_data_dist["Off"][:, :, i, j])) for i in range(num_subjects)])
    null_clustering_coef_Off = np.array([np.mean(bct.clustering_coef_bu(null_data_dist["Off"][:, :, i, j])) for i in range(num_subjects)])
    network_clustering_coef_On = np.array([np.mean(bct.clustering_coef_wu(thresholded_data_dist["On"][:, :, i, j])) for i in range(num_subjects)])
    null_clustering_coef_On = np.array([np.mean(bct.clustering_coef_bu(null_data_dist["On"][:, :, i, j])) for i in range(num_subjects)])
    network_measures_wei_Off["norm_clustering_coef"][j][:] = network_clustering_coef_Off / null_clustering_coef_Off
    network_measures_wei_On["norm_clustering_coef"][j][:] = network_clustering_coef_On / null_clustering_coef_On
    
    # small-worldness
    network_measures_wei_Off["small_worldness"][j][:] = network_measures_wei_Off["norm_clustering_coef"][j] / network_measures_wei_Off["norm_charact_path_length"][j]
    network_measures_wei_On["small_worldness"][j][:] = network_measures_wei_On["norm_clustering_coef"][j] / network_measures_wei_On["norm_charact_path_length"][j] 
    
    # Size of the Largest Connected Component (LCC)
    network_measures_wei_Off["size_of_LCC"][j][:] = np.array([len(max(nx.connected_components(nx.from_numpy_array(thresholded_data_dist["Off"][:, :, i, j])))) for i in range(num_subjects)])
    network_measures_wei_On["size_of_LCC"][j][:] = np.array([len(max(nx.connected_components(nx.from_numpy_array(thresholded_data_dist["On"][:, :, i, j])))) for i in range(num_subjects)])
    
    # Clustering coefficient saved for all nodes
    centrality_net_clust_Off = np.array([bct.clustering_coef_wu(thresholded_data_dist["Off"][:, :, i, j]) for i in range(num_subjects)])
    centrality_net_clust_On = np.array([bct.clustering_coef_wu(thresholded_data_dist["On"][:, :, i, j]) for i in range(num_subjects)])
    # circumventing the issue of having a 0 clustering coefficient for a given region by dividing by the mean clustering coefficent (over all regions) 
    # of the null-hypothesis graph for a given graph density and subject. 
    centrality_measures_wei_Off["clustering_coef"][j][:][:] = centrality_net_clust_Off #/ null_clustering_coef_Off[:, np.newaxis]
    centrality_measures_wei_On["clustering_coef"][j][:][:] = centrality_net_clust_On #/ null_clustering_coef_On[:, np.newaxis]
    
    # Degree for all nodes
    centrality_net_deg_Off = np.array([bct.degrees_und(thresholded_data_dist["Off"][:, :, i, j]) for i in range(num_subjects)])
    centrality_net_deg_On = np.array([bct.degrees_und(thresholded_data_dist["On"][:, :, i, j]) for i in range(num_subjects)])
    null_mean_deg_Off = np.array([np.mean(bct.degrees_und(null_data_dist["Off"][:, :, i, j])) for i in range(num_subjects)])
    null_mean_deg_On = np.array([np.mean(bct.degrees_und(null_data_dist["On"][:, :, i, j])) for i in range(num_subjects)])
    centrality_measures_wei_Off["degree"][j][:][:] = centrality_net_deg_Off / null_mean_deg_Off[:, np.newaxis]
    centrality_measures_wei_On["degree"][j][:][:] = centrality_net_deg_On / null_mean_deg_On[:, np.newaxis]
"""

  0%|          | 0/9 [00:00<?, ?it/s]

In [1]:
"""
for j in tqdm(range(num_thresholds)): 
    # Local Efficiency (weighted networks)
    network_measures_wei_Off["local_eff"][j][:][:] = local_eff(thresholded_data_dist["Off"][:, :, :, j], num_subjects)
    network_measures_wei_Off["local_eff"][j][:][:] = local_eff(thresholded_data_dist["On"][:, :, :, j], num_subjects)
"""

NameError: name 'tqdm' is not defined

In [ ]:
"""
for j in tqdm(range(num_thresholds)): 
    # Local efficiency for all nodes (weighted networks)
    centrality_net_local_eff_wei_Off = np.array([[local_efficiency_node(nx.from_numpy_array(thresholded_data_dist["Off"][:, :, i, j]), node) 
                                              for node in nx.from_numpy_array(thresholded_data_dist["Off"][:, :, i, j]).nodes ] 
                                              for i in range(num_subjects)])
    centrality_net_local_eff_wei_On = np.array([[local_efficiency_node(nx.from_numpy_array(thresholded_data_dist["On"][:, :, i, j]), node) 
                                              for node in nx.from_numpy_array(thresholded_data_dist["On"][:, :, i, j]).nodes] 
                                              for i in range(num_subjects)])
    centrality_measures_wei_Off["local_eff"][j][:][:] = centrality_net_local_eff_Off #/ local_eff_null_Off[:, np.newaxis]
    centrality_measures_wei_On["local_eff"][j][:][:] = centrality_net_local_eff_On #/ local_eff_null_On[:, np.newaxis]  
"""

---
---

# 2. Saving and Loading network measures

## 2.1. Saving network measures to .txt files

In [41]:
import json

# Convert numpy arrays to lists to make them JSON serializable
for key, value in network_measures_Off.items():
    if isinstance(value, np.ndarray):
        network_measures_Off[key] = value.tolist()
for key, value in network_measures_On.items():
    if isinstance(value, np.ndarray):
        network_measures_On[key] = value.tolist()
for key, value in centrality_measures_Off.items():
    if isinstance(value, np.ndarray):
        centrality_measures_Off[key] = value.tolist()
for key, value in centrality_measures_On.items():
    if isinstance(value, np.ndarray):
        centrality_measures_On[key] = value.tolist()

# dump the dictionary in a new file
with open('network_measures_Off.txt', 'w') as f:
    f.write(json.dumps(network_measures_Off, indent=4))
with open('network_measures_On.txt', 'w') as f:
    f.write(json.dumps(network_measures_On, indent=4))
with open('centrality_measures_Off.txt', 'w') as f:
    f.write(json.dumps(centrality_measures_Off, indent=4))
with open('centrality_measures_On.txt', 'w') as f:
    f.write(json.dumps(centrality_measures_On, indent=4))

In [69]:
"""
import json

# Same thing for weighted networks
for key, value in network_measures_wei_Off.items():
    if isinstance(value, np.ndarray):
        network_measures_wei_Off[key] = value.tolist()
for key, value in network_measures_wei_On.items():
    if isinstance(value, np.ndarray):
        network_measures_wei_On[key] = value.tolist()
for key, value in centrality_measures_wei_Off.items():
    if isinstance(value, np.ndarray):
        centrality_measures_wei_Off[key] = value.tolist()
for key, value in centrality_measures_wei_On.items():
    if isinstance(value, np.ndarray):
        centrality_measures_wei_On[key] = value.tolist()

with open('network_measures_wei_Off.txt', 'w') as f:
    f.write(json.dumps(network_measures_wei_Off, indent=4))
with open('network_measures_wei_On.txt', 'w') as f:
    f.write(json.dumps(network_measures_wei_On, indent=4))
with open('centrality_measures_wei_Off.txt', 'w') as f:
    f.write(json.dumps(centrality_measures_wei_Off, indent=4))
with open('centrality_measures_wei_On.txt', 'w') as f:
    f.write(json.dumps(centrality_measures_wei_On, indent=4))
"""

## 2.2. Loading global network measures

In [12]:
import json

# Load the data back from the file
with open('network_measures_Off.txt', 'r') as f:
    network_measures_Off = json.load(f)
with open('network_measures_On.txt', 'r') as f:
    network_measures_On = json.load(f)
with open('centrality_measures_Off.txt', 'r') as f:
    centrality_measures_Off = json.load(f)
with open('centrality_measures_On.txt', 'r') as f:
    centrality_measures_On = json.load(f)

# Convert lists back to numpy arrays
for key, value in network_measures_Off.items():
    network_measures_Off[key] = np.array(value)
for key, value in network_measures_On.items():
    network_measures_On[key] = np.array(value)  
for key, value in centrality_measures_Off.items():
    centrality_measures_Off[key] = np.array(value)  
for key, value in centrality_measures_On.items():
    centrality_measures_On[key] = np.array(value)

In [13]:
"""
import json

# Same thing for weighted networks
with open('network_measures_wei_Off.txt', 'r') as f:
    network_measures_wei_Off = json.load(f)
with open('network_measures_wei_On.txt', 'r') as f:
    network_measures_wei_On = json.load(f)
with open('centrality_measures_wei_Off.txt', 'r') as f:
    centrality_measures_wei_Off = json.load(f)
with open('centrality_measures_wei_On.txt', 'r') as f:
    centrality_measures_wei_On = json.load(f)

for key, value in network_measures_wei_Off.items():
    network_measures_wei_Off[key] = np.array(value)
for key, value in network_measures_wei_On.items():
    network_measures_wei_On[key] = np.array(value)  
for key, value in centrality_measures_wei_Off.items():
    centrality_measures_wei_Off[key] = np.array(value)  
for key, value in centrality_measures_wei_On.items():
    centrality_measures_wei_On[key] = np.array(value)
"""

FileNotFoundError: [Errno 2] No such file or directory: 'centrality_measures_wei_Off.txt'

Shape Check:

In [ ]:
print(network_measures_Off["norm_charact_path_length"].shape)
print(network_measures_wei_Off["norm_charact_path_length"].shape)

(45, 104)
(5, 104)
(9, 104)
(5, 104)
(5,)
